In [1]:
from preprocess import create_documents, chunk_documents

In [2]:
#filename = 'Omega-cert-v1-PD_ON-2023-11-21'
#filename = 'Omega-cert-v1-startup-and-SOL-2023-11-21'
filename = 'Omega-cert-v1-startup-no-TP-2023-11-21'

docs = create_documents(f"logs/dlt/{filename}.dlt")
chunks = chunk_documents(docs, chunk_size=8000)

In [3]:
len(docs), len(chunks)

(316, 3)

In [4]:
docs

['START HPA QSYS SLOG 4100\n1 error: libnvtegrahv: Failed to open rsmgr file: : MASK\nEND',
 'START HPA QSYS SLOG 8204\n1 error: NVMAP Init Start\n1 error: Found IVM pools: : MASK\n1 error: NVMAP Init Completed\nEND',
 'START HPA QSYS SLOG 8212\n1 error: Did not find property mac_speed_mbps. Assuming mac_speed_mbps = 1000Mbps\n1 fatal: preprocess_server_init: server_map[0] - prefix  typed_memory /memory/eqosMemory client_prealloc_max 200 uid_allow 3220 gid_allow 3229 gid_default 3220\nEND',
 'START HPA QSYS SLOG 32790\n1 error: Enabling VSCD optimizations\nEND',
 'START HPA QSYS SLOG 32791\n1 error: Enabling VSCD optimizations\nEND',
 'START HPA QSYS SLOG 32792\n1 error: Enabling VSCD optimizations\nEND',
 'START HPA QSYS SLOG 32794\n1 error: Enabling VSCD optimizations\nEND',
 'START HPA QSYS SLOG 32795\n1 error: Enabling VSCD optimizations\nEND',
 'START HPA QSYS SLOG 61469\n2 error: set resets from DT failed for:\n1 error: vi-thi\n1 error: isp-thi\n352 error: Lost receive id: MASK\n

In [5]:
from models.llms import LLMFactory

llm = LLMFactory.create('azure', engine='gpt-4')


                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [6]:
from utils.templates import SystemPromptMappingTemplate, SystemPromptConbineTemplate
map_prompt_template = SystemPromptMappingTemplate()
combine_prompt_template = SystemPromptConbineTemplate()

In [7]:
from chains import get_summarization_chain
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    summary_chain = get_summarization_chain(llm=llm,
                                        map_prompt=map_prompt_template,
                                        reduce_prompt=combine_prompt_template,
                                        verbose=True)


    output = summary_chain.run(chunks)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a senior software engineer. Your task is to summarize logs. The logs given to you are divided into sessions following this format:
START [session title]
[occurrence count] [log type]: [payload]
END
Some numbers are replcaed with 'MASK' becasue they are not important.
Log to be summarized: {START HPA QSYS SLOG 4100
1 error: libnvtegrahv: Failed to open rsmgr file: : MASK
END
START HPA QSYS SLOG 8204
1 error: NVMAP Init Start
1 error: Found IVM pools: : MASK
1 error: NVMAP Init Completed
END
START HPA QSYS SLOG 8212
1 error: Did not find property mac_speed_mbps. Assuming mac_speed_mbps = 1000Mbps
1 fatal: preprocess_server_init: server_map[0] - prefix  typed_memory /memory/eqosMemory client_prealloc_max 200 uid_allow 3220 gid_allow 3229 gid_default 3220
END
START HPA QSYS SLOG 32790
1 error: Enabling VSCD optimizations
END
START HPA QSYS SLOG 32791
1 error: Enabling VSCD 

# 5m 34.5s
# 51.7s
# 1m 27.9s
# 53s

In [8]:
print(output)

# Summary
The logs indicate multiple errors across various sessions. The most common errors include:

- Service unavailability, such as the AlcoholInterlockManager, Vehicle motion state service, Vehiclemodesprovider, Climateandthermalcontrolobserver, Actuator admin service, Vehicle Mode Provider, Electrical Energy, and OccupantPresenceProvider services. This could be due to services not being properly initialized or running.
- Device states not being found among device states from DeviceAdmin, which could indicate a problem with device registration or identification.
- Failed attempts to read or get certain properties or services, such as the 'Failed to get usage_mode_awake status' and 'Failed to read Day Night info from ambientlighthal', suggest potential issues with data access or retrieval.
- Device activation and device not found issues, particularly with devices OHTL, OHTR, and RLSM. These errors are primarily due to failed requests to the DeviceAdmin.
- Errors related to failed c

In [9]:
with open(f'{filename}.md','w') as f:
    f.write(output)
    f.write('\n\n')
    f.write(f'```\n{str(cb)}\n```')